# Checking Available Data and Creating Metadata for Replication

In [1]:
# Load packages
import pandas as pd
import numpy as np
from pyhere import here
import pygris
from pygris.data import get_census
from pygris import counties
pd.options.display.max_colwidth = 1000
pd.set_option("chained_assignment", None)

In [2]:
# Load reproduction data dictionary
acs_vars = pd.read_csv( here("data", "metadata", "ACS_2012_data_dictionary.csv") ) 

acs_vars.drop(columns=acs_vars.columns[0], axis=1, inplace=True)

rpl_vars = acs_vars[["Reproduction Label", "Alias", "Definition"]]

rpl_vars = rpl_vars.rename(columns={"Reproduction Label": "Label"})

variables = list(rpl_vars['Label'][1:])

aliases = list(rpl_vars['Alias'][1:])

## Checking Data Availability

In [3]:
html_list = [ # this includes all 5-year ACS data available
    "https://api.census.gov/data/2009/acs/acs5/variables.html",
    "https://api.census.gov/data/2010/acs/acs5/variables.html",
    "https://api.census.gov/data/2011/acs/acs5/variables.html",
    "https://api.census.gov/data/2012/acs/acs5/variables.html",
    "https://api.census.gov/data/2013/acs/acs5/variables.html",
    "https://api.census.gov/data/2014/acs/acs5/variables.html",
    "https://api.census.gov/data/2015/acs/acs5/variables.html",
    "https://api.census.gov/data/2016/acs/acs5/variables.html",
    "https://api.census.gov/data/2017/acs/acs5/variables.html",
    "https://api.census.gov/data/2018/acs/acs5/variables.html",
    "https://api.census.gov/data/2019/acs/acs5/variables.html",
    "https://api.census.gov/data/2020/acs/acs5/variables.html",
    "https://api.census.gov/data/2021/acs/acs5/variables.html"
]

ref_var_list = pd.read_html("https://api.census.gov/data/2012/acs/acs5/variables.html")[0]
ref_meta = pd.DataFrame( {"Name": variables,
                          "Alias": aliases} )
ref_meta = ref_meta.merge(ref_var_list, on = "Name", how = "left")[["Name", "Label", "Concept", "Alias"]]

ref_meta["Definition_reference"] = ref_meta["Concept"] + ' ' + ref_meta["Label"]
ref_meta["Label_reference"] = ref_meta["Label"]
ref_meta = ref_meta.drop( ["Concept", "Label"], axis = 1 )

ref_meta[['Definition_reference']] = ref_meta[['Definition_reference']].replace([r"(?<!\d)\d{4}(?!\d)", '!', ':', ' --'], '', regex=True)
ref_meta['Definition_reference'] = ref_meta['Definition_reference'].str.lower()
ref_meta[['Label_reference']] = ref_meta[['Label_reference']].replace([r"(?<!\d)\d{4}(?!\d)", '!', ':'], '', regex=True)
ref_meta['Label_reference'] = ref_meta['Label_reference'].str.lower()


for link in html_list:
    var_list = pd.read_html(link)[0]
    acs_meta = pd.DataFrame( {"Name": variables,
                              "Alias": aliases} )
    acs_meta = acs_meta.merge(var_list, on = "Name", how = "left")[["Name", "Label", "Concept", "Alias"]]
    
    
    acs_meta["Definition"] = acs_meta["Concept"] + ' ' + acs_meta["Label"]
    
    if acs_meta["Concept"].isnull().sum() == 45:
        acs_meta[['Label']] = acs_meta[['Label']].replace([r"(?<!\d)\d{4}(?!\d)", '!', ':'], '', regex=True)
        acs_meta['Label'] = acs_meta['Label'].str.lower()
    else:
        acs_meta[['Definition']] = acs_meta[['Definition']].replace([r"(?<!\d)\d{4}(?!\d)", '!', ':', ' --'], '', regex=True)
        acs_meta['Definition'] = acs_meta['Definition'].str.lower()

    joined_meta = acs_meta.merge(ref_meta, on = ["Name", "Alias"])
    
    if acs_meta["Concept"].isnull().sum() == 45:
        eq = ~joined_meta['Label_reference'].eq(joined_meta['Label'])
        issues = joined_meta.loc[eq][['Name', 'Alias', 'Label_reference', 'Label']]
    else:
        eq = ~joined_meta['Definition_reference'].eq(joined_meta['Definition'])
        issues = joined_meta.loc[eq][['Name', 'Alias', 'Definition_reference', 'Definition']]

    if len(issues) > 0:
        print("--------------------------------------------------------------------------------------------------------------------------------\nVariable definitions in the HTML link", link, "have the following discrepancies with variable definitions in the 2012 5-year ACS link:")
        display(issues)
    else:
        print("--------------------------------------------------------------------------------------------------------------------------------\nThe following link contains all the right vars:", link)

--------------------------------------------------------------------------------------------------------------------------------
Variable definitions in the HTML link https://api.census.gov/data/2009/acs/acs5/variables.html have the following discrepancies with variable definitions in the 2012 5-year ACS link:


,Name,Alias,Definition_reference,Definition
6,B06001_002E,total population under 5 years of age,place of birth by age in the united states estimatetotalunder 5 years,NaN
7,B09020_001E,total population over 65 years of age,relationship by household type (including living alone) for the population 65 years and over estimatetotal,NaN
13,B09020_021E,total 65+ living in group quarters,relationship by household type (including living alone) for the population 65 years and over estimatetotalin group quarters,NaN
26,C24010_038E,total female employed,sex by occupation for the civilian employed population 16 years and over estimatetotalfemale,"sex by occupation for the civilian employed population 16 years and over estimatetotalmaleproduction, transportation, and material moving occupationstransportation and material moving occupationsmaterial moving workers"
33,B06007_005E,total Spanish-speakers who speak english less than very well,"place of birth by language spoken at home and ability to speak english in the united states estimatetotalspeak spanishspeak english less than ""very well""",NaN
34,B06007_008E,total people who speak another language and speak English less than very well,"place of birth by language spoken at home and ability to speak english in the united states estimatetotalspeak other languagesspeak english less than ""very well""",NaN
35,B06007_001E,total population with known language spoken at home and English ability,place of birth by language spoken at home and ability to speak english in the united states estimatetotal,NaN
36,B16010_002E,total population with less than a high school graduate education,educational attainment and employment status by language spoken at home for the population 25 years and over estimatetotalless than high school graduate,NaN
37,B16010_001E,"total for which education, employment, language at home known",educational attainment and employment status by language spoken at home for the population 25 years and over estimatetotal,NaN
38,C24050_002E,total population in extractive industries,"industry by occupation for the civilian employed population 16 years and over estimatetotalagriculture, forestry, fishing and hunting, and mining",NaN


--------------------------------------------------------------------------------------------------------------------------------
Variable definitions in the HTML link https://api.census.gov/data/2010/acs/acs5/variables.html have the following discrepancies with variable definitions in the 2012 5-year ACS link:


,Name,Alias,Definition_reference,Definition
7,B09020_001E,total population over 65 years of age,relationship by household type (including living alone) for the population 65 years and over estimatetotal,NaN
13,B09020_021E,total 65+ living in group quarters,relationship by household type (including living alone) for the population 65 years and over estimatetotalin group quarters,NaN


--------------------------------------------------------------------------------------------------------------------------------
Variable definitions in the HTML link https://api.census.gov/data/2011/acs/acs5/variables.html have the following discrepancies with variable definitions in the 2012 5-year ACS link:


...

,Name,Alias,Definition_reference,Definition
15,B11001_006E,total female-headed family households,"household type (including living alone) estimatetotalfamily householdsother familyfemale householder, no husband present","household type (including living alone) estimatetotalfamily householdsother familyfemale householder, no spouse present"


--------------------------------------------------------------------------------------------------------------------------------
Variable definitions in the HTML link https://api.census.gov/data/2020/acs/acs5/variables.html have the following discrepancies with variable definitions in the 2012 5-year ACS link:


,Name,Alias,Definition_reference,Definition
15,B11001_006E,total female-headed family households,"household type (including living alone) estimatetotalfamily householdsother familyfemale householder, no husband present","household type (including living alone) estimatetotalfamily householdsother familyfemale householder, no spouse present"


--------------------------------------------------------------------------------------------------------------------------------
Variable definitions in the HTML link https://api.census.gov/data/2021/acs/acs5/variables.html have the following discrepancies with variable definitions in the 2012 5-year ACS link:


,Name,Alias,Definition_reference,Definition
15,B11001_006E,total female-headed family households,"household type (including living alone) estimatetotalfamily householdsother familyfemale householder, no husband present","household type (including living alone) estimatetotalfamily householdsother familyfemale householder, no spouse present"


Notice that the variables for 5-year ACS data 2012-2021 are all perfect matches for the variables in the 2012 5-year ACS used in Spielman et al.'s original study (the only difference is that in 2019-2021 they changed "no husband present" to "no spouse present" for total female-headed family households).
For this reason, we will use 5-year ACS data for each year from 2012-2021 in our study.

## Creating Data Dictionary Files

In [4]:
# Create overarching data dictionary (will print this in replication report) 
rpl_vars.to_csv( here("data", "metadata", "replication_vars.csv") )

In [6]:
# Create more detailed data dictionaries (will link to this in replication report)
data = {}
for i in range(2012, 2022):    
    # Obtain data
    data["counties"+str(i)] = get_census(dataset = "acs/acs5", # dataset name on the Census API you are connecting to; find datasets at https://api.census.gov/data.html
                                         variables = variables, # string (or list of strings) of desired vars. For the 2021 5-year ACS Data Profile, those variable IDs are found at https://api.census.gov/data/2021/acs/acs5/profile/variables.html
                                         year = i, # year of your data (or end-year for a 5-year ACS sample)
                                         params = { # dict of query parameters to send to the API.
                                             "for": "county:*"},
                                         guess_dtypes = True,
                                         return_geoid = True)

    # Drop Puerto Rico
    data["counties"+str(i)] = data["counties"+str(i)].loc[~data["counties"+str(i)]['GEOID'].str.startswith('72')]


    acs_meta = rpl_vars.iloc[1:].reset_index().drop(columns={'index'})

    # Create data type column
    acs_meta["Type"] = np.NaN

    # Create domain column
    acs_meta["Domain"] = np.NaN

    # Create missing data column
    acs_meta["Missing Data Value(s)"] = "NaN"

    # Create missing data frequency
    acs_meta["Missing Data Frequency"] = np.NaN

    for j in range(len(variables)):
        var = acs_meta["Label"][j]

        acs_meta["Type"][j] = data["counties"+str(i)][var].dtype

        var_min = data["counties"+str(i)][var].min()
        var_max = data["counties"+str(i)][var].max()

        if var_min%1 == 0:
            var_min = round(var_min)
        if var_max%1 == 0:
            var_max = round(var_max)

        acs_meta["Domain"][j] = str(var_min) + " - " + str(var_max)

        acs_meta["Missing Data Frequency"][j] = np.isnan(data["counties"+str(i)][var]).sum()

    geoid = pd.DataFrame(
    [["GEOID", 'FIPS code unique identifier', 'Unique code for every county and county-equivalent in USA', 'string', '01001 - 56045', 'None', 0]],
    columns= acs_meta.columns)

    acs_meta = pd.concat([geoid,acs_meta])

    acs_meta.to_csv('RPl_ACS_' + str(i) + '_data_dictionary.csv')

## Creating Metadata Files